# Proyecto Final: Clasificación de tópicos de interés

## Exploración de datos


CC5113 - Aprendizaje Automático Bayesiano

Profesor: Pablo Guerrero

Autor: Martín Cornejo

## Paquetes necesarios

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
import itertools
import operator

import pandas as pd
from nltk.stem.snowball import SpanishStemmer

In [2]:
#data = np.genfromtxt('data_format.csv', delimiter='')
datos=pd.read_csv('data_format.csv')
print(datos.shape)
#pdb.set_trace()
datos.head()

(138, 2)


,Texto,Interes
0,"Ojalá obliguen a Piñera a cerrar Punta Peuco, ...",False
1,Piñera para crear base de apoyo moderada a su ...,True
2,@CNNChile MEMORIA 2014 Adimark: Piñera termina...,True
3,PPK y Piñera en privado habrían conversado alg...,False
4,Bachelet entregará el gobierno de Chile a Piñera,True


## Limpiando strings

### Reemplazar tildes, caracteres especiales, todo a minúsculas

In [3]:
strings = datos.ix[:,0]
tildes = ['á','é','í','ó','ú']
vocales = ['a','e','i','o','u']

# tildes
for idx, vocal in enumerate(vocales):
    strings = strings.str.replace(tildes[idx],vocal)

# caracteres especiales menos la ñ
strings = strings.str.replace('[^a-zñA-Z ]', "")

# todo a minusculas
strings = pd.Series(list(map(lambda x: x.lower(), strings)))
print(strings.shape)
strings.head()

(138,)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


0    ojala obliguen a piñera a cerrar punta peuco e...
1    piñera para crear base de apoyo moderada a su ...
2    cnnchile memoria  adimark piñera termina su go...
3    ppk y piñera en privado habrian conversado alg...
4     bachelet entregara el gobierno de chile a piñera
dtype: object

## Stemming

In [41]:
stemmer = SpanishStemmer()

# oracion a arreglo de strings
strings_arr = list(map(lambda x: x.split(), strings))
strings_arr = list(itertools.chain.from_iterable(strings_arr))
root_arr = list(map(lambda x: stemmer.stem(x), strings_arr))

def most_common(str_arr):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(str_arr))
  list_pairs = []
  #print('SL:', SL)
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
    
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(str_arr)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    list_pairs.append((item, count))
    #print('item %r, count %r, minind %r' % (item, count, min_index))
    return count, -min_index

  return max(groups, key=_auxfun)[0], list_pairs

stem_common, list_pairs = most_common(root_arr)

pares_filtrados = list(filter(lambda x: len(x[0]) > 2, list_pairs))

common_roots_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)

## Lematización

In [45]:
def create_lemma_dict(filename):
   with open(filename, 'r') as document:
       lemma_dict = {}
       for line in document:
           if line.strip():  # avoid empty lines
               value, key = line.split(None, 1) # 'None' means 'all whitespace', which is the default
               key = key.rstrip() # rstrip() to get rid of \r and \n
               lemma_dict[key] = value # adding the flections as keys to the dict
               lemma_dict[value] = value # adding also the base word as a key
   return lemma_dict

def query_word(lemma_dict):
   word = input("\nDame una palabra en español -> ")
   try:
      lemma = lemma_dict[word]
      print("__your happy lemma is__: {}".format(lemma))
   except KeyError:
      print("This word is not in the dictionary!")
   return query_word(lemma_dict)

def lemmatiser(dict):
    def lookup(word):
        try:
            lemma = dict[word]
        except:
            lemma = word
        
        return lemma
    
    return lookup

resource_file = 'lemmatization-es.txt'
lemmatiser_es = lemmatiser(create_lemma_dict(resource_file))
#pdb.set_trace()

lemma_arr = list(map(lemmatiser_es, strings_arr))
common_lemma, pairs_lemma = most_common(lemma_arr)
pares_filtrados = list(filter(lambda x: len(x[0]) > 3, pairs_lemma))
common_lemma_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)
common_lemma_sorted

[('piñera', 133),
 ('sebastian', 30),
 ('chile', 24),
 ('este', 24),
 ('presidente', 24),
 ('bachelet', 23),
 ('comer', 19),
 ('parir', 17),
 ('gobierno', 16),
 ('haber', 11),
 ('mando', 9),
 ('peuco', 9),
 ('tener', 9),
 ('ahora', 8),
 ('bien', 8),
 ('cerrar', 8),
 ('llegar', 8),
 ('mañana', 8),
 ('punta', 8),
 ('reunion', 8),
 ('todo', 8),
 ('cuando', 7),
 ('decir', 7),
 ('macri', 7),
 ('nuevo', 7),
 ('actividad', 6),
 ('asumir', 6),
 ('cambio', 6),
 ('cierre', 6),
 ('domingo', 6),
 ('estar', 6),
 ('pais', 6),
 ('poder', 6),
 ('querer', 6),
 ('asuncion', 5),
 ('cierrepuntapeuco', 5),
 ('dejar', 5),
 ('derecho', 5),
 ('piera', 5),
 ('poner', 5),
 ('sera', 5),
 ('tiempo', 5),
 ('bueno', 4),
 ('carlos', 4),
 ('chileno', 4),
 ('deber', 4),
 ('entrar', 4),
 ('ganar', 4),
 ('juan', 4),
 ('mandato', 4),
 ('mejor', 4),
 ('oficial', 4),
 ('sename', 4),
 ('sobre', 4),
 ('temer', 4),
 ('terminar', 4),
 ('tomar', 4),
 ('venezuela', 4),
 ('votar', 4),
 ('academia', 3),
 ('algo', 3),
 ('asistir', 